In [10]:
# from os import getenv
from connect_db import connect_db
# con = connect_db(
#     username=getenv("ORACLE_USERNAME"),
#     password=getenv("ORACLE_PASSWORD"),
#     host=getenv("ORACLE_HOST"),
#     port=int(getenv("ORACLE_PORT"))
# )
con = connect_db(
    username="database",
    password="database",
    host="localhost",
    port=1521
)
cur = con.cursor()

In [22]:
A_CHECK_PER_MONTHS = 1
A_CHECK_DURATION_HOURS = 10
B_CHECK_PER_MONTHS = 6
B_CHECK_DURATION_HOURS = 2 * 24
C_CHECK_PER_MONTHS = 24
C_CHECK_DURATION_HOURS = 10 * 24

In [23]:
from datetime import datetime
from typing import NamedTuple
from enum import Enum, auto

class MaintenanceType(Enum):
    ACHK = auto()
    BCHK = auto()
    CCHK = auto()
    DCHK = auto()

class Maintenance(NamedTuple):
    aircraft_id: str
    maintenance_type: MaintenanceType
    dt: datetime
    description: str

In [24]:
from datetime import timedelta
import random

def get_timedelta_in_hours(td: timedelta) -> float:
    return td.total_seconds() / 60 / 60

def get_timedelta_in_months(td: timedelta, days_per_months = 30) -> float:
    return get_timedelta_in_hours(td) / 24 / days_per_months

# https://stackoverflow.com/questions/553303/generate-a-random-date-between-two-other-dates
def random_date(start: datetime, end: datetime):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = random.randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [25]:
from dataclasses import dataclass

@dataclass(slots=True)
class CheckDetail:
    a_check: datetime
    b_check: datetime
    c_check: datetime
    
    def get_last_check(self, maintenance_type: MaintenanceType) -> datetime:
        match maintenance_type:
            case MaintenanceType.ACHK:
                return max(self.a_check, self.b_check, self.c_check)
            case MaintenanceType.BCHK:
                return max(self.b_check, self.c_check)
            case MaintenanceType.CCHK:
                return self.c_check

stmt = "SELECT AIRCRAFT_ID, PURCHASE_DATE FROM AIRCRAFT"
AIRCRAFT_LAST_CHECK: dict[str, CheckDetail] = {}
for row in cur.execute(stmt):
    AIRCRAFT_LAST_CHECK[row[0]] = CheckDetail(
        a_check=random_date(max(row[1], datetime(2022, 12, 1)), datetime(2023, 1, 1)),
        b_check=random_date(max(row[1], datetime(2022, 6, 1)), datetime(2023, 1, 1)),
        c_check=random_date(max(row[1], datetime(2021, 1, 1)), datetime(2023, 1, 1))
    )

In [26]:
def schedule_next_maintenance(aircraft_id: str, idle_time: datetime, duration_in_hours: float) -> Maintenance:
    check = AIRCRAFT_LAST_CHECK[aircraft_id]
    if get_timedelta_in_months(idle_time - check.get_last_check(MaintenanceType.CCHK)) > C_CHECK_PER_MONTHS:
        if duration_in_hours > C_CHECK_DURATION_HOURS:
            return Maintenance(
                aircraft_id=aircraft_id,
                maintenance_type=MaintenanceType.CCHK,
                dt=idle_time,
                description=""
            )
    if get_timedelta_in_months(idle_time - check.get_last_check(MaintenanceType.BCHK)) > B_CHECK_PER_MONTHS:
        if duration_in_hours > B_CHECK_DURATION_HOURS:
            return Maintenance(
                aircraft_id=aircraft_id,
                maintenance_type=MaintenanceType.BCHK,
                dt=idle_time,
                description=""
            )
    if get_timedelta_in_months(idle_time - check.get_last_check(MaintenanceType.ACHK)) > A_CHECK_PER_MONTHS:
        if duration_in_hours > A_CHECK_DURATION_HOURS:
            return Maintenance(
                aircraft_id=aircraft_id,
                maintenance_type=MaintenanceType.ACHK,
                dt=idle_time,
                description=""
            )
        
def update_maintenance(maintenance: Maintenance):
    match maintenance.maintenance_type:
        case MaintenanceType.ACHK:
            AIRCRAFT_LAST_CHECK[maintenance.aircraft_id].a_check = maintenance.dt
        case MaintenanceType.BCHK:
            AIRCRAFT_LAST_CHECK[maintenance.aircraft_id].b_check = maintenance.dt
        case MaintenanceType.CCHK:
            AIRCRAFT_LAST_CHECK[maintenance.aircraft_id].c_check = maintenance.dt

In [ ]:
from utils import paginate_insert_all

sql = "    INTO MAINTENANCE_LOG (AIRCRAFT_ID, MAINTENANCE_TYPE_ID, MAINTENANCE_DATETIME, MAINTENANCE_DESCRIPTION) VALUES ('{}', '{}', TO_TIMESTAMP('{:%Y-%m-%d %H:%M:%S}', 'YYYY-MM-DD HH24:MI:SS'), '{}')"
@paginate_insert_all
def insert_maintenance(maintenance: Maintenance):
    print(sql.format(maintenance.aircraft_id, maintenance.maintenance_type.name, maintenance.dt, maintenance.description))

stmt = """
SELECT
    AIRCRAFT_ID,
    LAG(DEPARTURE_DATETIME) OVER (ORDER BY AIRCRAFT_ID, DEPARTURE_DATETIME) AS "Last Departure",
    LAG(DEPARTURE_DATETIME + EST_DURATION_IN_HOUR / 24) OVER (ORDER BY AIRCRAFT_ID, DEPARTURE_DATETIME) AS "Expected Arrival / Idle Start",
    DEPARTURE_DATETIME AS "Next Departure Time",
    DEPARTURE_DATETIME - LAG(DEPARTURE_DATETIME + EST_DURATION_IN_HOUR / 24) OVER (ORDER BY AIRCRAFT_ID, DEPARTURE_DATETIME) AS "Idle Duration"
FROM FLIGHT
ORDER BY AIRCRAFT_ID, DEPARTURE_DATETIME
"""
for row in cur.execute(stmt):
    if not row[4]:
        continue
    
    aircraft_id: str = row[0]
    idle_start: datetime = row[2]
    duration: timedelta = row[4]
    
    idle_start += timedelta(hours=1)
    duration -= timedelta(hours=1)
    
    maintenance = schedule_next_maintenance(aircraft_id, idle_start, get_timedelta_in_hours(duration))
    if not maintenance:
        continue
    
    insert_maintenance(maintenance)
        
    update_maintenance(maintenance)
    
print("SELECT 1 FROM DUAL;")

In [28]:
con.close()